In [ ]:
import numpy as np
from plotly import graph_objs as go
from scipy import fft

In [ ]:
fs_hz = 48e3
signal_duration_sec = 0.1
t = np.arange(0, signal_duration_sec, 1/fs_hz)

print(f"Number of samples: {len(t)}")

In [ ]:
def calc_spectrum(sig, fs=fs_hz):
    S = 20*np.log10(np.abs(fft.rfft(sig)))
    f = fft.rfftfreq(len(sig), d=1/fs)

    return f, S

In [ ]:
def saturationFunc(sig):
    return np.tanh(sig)

x = np.linspace(-3, 3)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=saturationFunc(x), name='saturationFunc'))
fig.show()

In [ ]:
tone_freq_hz = 4.3e3
mag = 0.1
sig = mag * np.sin(2*np.pi * tone_freq_hz * t)

f, S = calc_spectrum(sig)
_, nlS = calc_spectrum(saturationFunc(sig))

fig = go.Figure()
fig.add_trace(go.Scatter(x=f, y=S, name='Before'))
fig.add_trace(go.Scatter(x=f, y=nlS, line=dict(width=2, dash='dash'), name='After'))
fig.update_layout(title='Nonlinearity impact',
                  xaxis_title='Frequency, Hz',
                  yaxis_title='Spectral density')
fig.show()

In [ ]:
mag = 0.1

fig = go.Figure()
freq_points_hz = np.arange(300, fs_hz/2, 200)
for tone_freq_hz in freq_points_hz:
    sig = mag * np.sin(2*np.pi * tone_freq_hz * t)
    f, S = calc_spectrum(sig)
    _, nlS = calc_spectrum(saturationFunc(sig))
    
    fig.add_trace(go.Scatter(x=f, y=S,
            visible=False, line=dict(color='blue'),
            name="Before"))
    
    fig.add_trace(go.Scatter(x=f, y=nlS,
            visible=False, line=dict(color='red', width=2, dash='dash'),
            name="After"))

active_ind = 1
fig.data[2*active_ind].visible = True
fig.data[2*active_ind+1].visible = True

steps = []
for i in range(len(freq_points_hz)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": f"Frequency: {freq_points_hz[i]} Hz"}],
        label=str(int(freq_points_hz[i]))
    )
    step["args"][0]["visible"][2*i] = True
    step["args"][0]["visible"][2*i+1] = True
    steps.append(step)

sliders = [dict(
    active=active_ind,
    currentvalue={"prefix": "Frequency: ", "suffix": " Hz"},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title=f"Signal frequency: {freq_points_hz[active_ind]} Hz"
)

fig.show()